In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# default_exp optionsseasonal

# optionsseasonal

> seasonal equity performance is used to guestimate performance probability

In [ ]:
#hide
from nbdev.showdoc import *
from hesap.pricedata import get_history_yf, get_history
from hesap.config import get_config

In [ ]:
#export

"""This module contains methods to create and update daily price and weekly
performance for the desired assets locally. Also for a given week, the most
optimal assets (with highest performance probability based on the history)
are identified for possible MattChoi's type option strategy.
by faf
Copyright 2021 Fafs
License: http://creativecommons.org/licenses/by/4.0/
"""

import os

import numpy as np
import pandas as pd

from typing import List


def weekly_returns(df):
    """
    Processes the daily adjusted close prices to produce 35 to 45 week forward
    performances
    """
    df.loc[:, 'year'] = df.index.isocalendar().year
    df.loc[:, 'week'] = pd.Int64Index(df.index.isocalendar().week)
    for n in range(35, 46):
        df.loc[:, f'forward{n}'] = df.loc[:, 'adj_close'].shift(-n) / df.loc[:, 'adj_close']

    rets, forwards, means = {}, {}, []
    for week in range(2, 52):
        if week not in df.week.unique(): continue
        forwards[week] = df.loc[df.week == week,
                                ['forward35', 'forward36', 'forward37',
                                 'forward38', 'forward39', 'forward40',
                                 'forward41', 'forward42', 'forward43',
                                 'forward44', 'forward45', 'year']]

        years = sorted(forwards[week].year.unique())
        rets[week] = []
        for year in years:
            returns = []
            for n in range(35,46):
                returns.extend(forwards[week].loc[(forwards[week].year == year), f'forward{n}'].tolist())
            rets[week].append({
                'year': year,
                'min': min(returns),
                'max': max(returns),
                'mean': np.mean(returns),
                'std': np.std(returns),
                'up_conf': np.sum([1 for r in returns if r > 1]) / len(returns) * 100,
                'dn_conf': np.sum([1 for r in returns if r < 1]) / len(returns) * 100
            })
        rets[week] = pd.DataFrame(rets[week])

        up_mean = round(rets[week].up_conf.mean(), 2)
        dn_mean = round(rets[week].dn_conf.mean(), 2)
        means.append({'week': week, 'up_mean': up_mean, 'dn_mean': dn_mean})
        if up_mean > 80 or dn_mean > 80:
            print(f'{week:2}:\t{up_mean}\t{dn_mean}')
    return rets, means


def build_weekly_db(tickers: List[str],
                    db_path: str,
                    start_date: str = '2000-01-01'
                   ) -> None:
    if tickers is None or len(tickers) == 0: return
    for d in [db_path, f'{db_path}/prices', f'{db_path}/rets',
              f'{db_path}/mean_rets']:
        os.makedirs(d, exist_ok=True)
    for i, ticker in enumerate(tickers):
        prices = get_history_yf(ticker.replace('.', '-'),
                                start_date=start_date)
        prices.to_csv(f'{db_path}/prices/{ticker}.csv')
        print(f'={i+1:4}{"="*4}{ticker:4}{"="*20}')
        rets, means = weekly_returns(prices)

        try:
            pd.DataFrame.from_dict(data=rets, orient='index').to_csv(f'{db_path}/rets/{ticker}.csv', header=False)
            pd.DataFrame(means).to_csv(f'{db_path}/mean_rets/{ticker}.csv')
        except:
            print(f'could not create dataframe for {ticker}')

In [ ]:
#hide
cfg = get_config('../my_secrets/options_weekly.yml')
for k in cfg:
    print(f'{k}:\t{cfg[k]}')
tickers = pd.read_csv(cfg['ticker_path'], sep='\t').loc[:, 'ticker'].to_list()
print(tickers)
# build_weekly_db(tickers, cfg['db_path'], '1990-01-01')

db_path:	/Users/fahrisurucu/Projects/my_data/
ticker_path:	/Users/fahrisurucu/Projects/my_secrets/sp500_20210613.tsv
['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BK', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'BIIB', 'BLK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'CHRW', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BF.B', 'CHRW', 'COG', 'CDNS', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CERN', 'CF', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'K

In [ ]:
prices = get_history_yf('VNT')
prices.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,adj_close,Volume
Date,,,,,,
2020-09-24,36.00,36.0,34.00,34.0,33.975784,743500
2020-09-28,35.00,35.0,33.00,33.0,32.976494,150500
2020-09-29,30.50,30.5,30.50,30.5,30.478275,220200
2020-09-30,30.75,31.0,30.75,31.0,30.977919,21900
2020-10-01,34.00,34.0,34.00,34.0,33.975784,43600
